In [1]:
import pandas as pd
from pathlib import Path
import os

In [2]:
input_path = Path.home() / "hi-paris-2025/data/raw/X_test.csv"

print("Fichier X_test :", input_path)
print("Existe :", input_path.is_file())

if input_path.is_file():
    size_bytes = os.path.getsize(input_path)
    size_gb = size_bytes / (1024**3)
    print(f"Taille de X_test.csv : {size_gb:.2f} Go")
else:
    raise FileNotFoundError(f"Fichier introuvable : {input_path}")

Fichier X_test : /mnt/hdd/homes/akok/hi-paris-2025/data/raw/X_test.csv
Existe : True
Taille de X_test.csv : 0.54 Go


In [4]:
chunksize = 10_000  

null_counts = None 
total_rows = 0      

for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):

    total_rows += len(chunk)
    chunk_nulls = chunk.isna().sum()
    
    if null_counts is None:
        null_counts = chunk_nulls
    else:
        null_counts = null_counts.add(chunk_nulls, fill_value=0)
    
    print(f"Chunk {i+1} traité, lignes dans ce chunk : {len(chunk)}")

print("Analyse terminée.")
print("Nombre total de lignes lues :", total_rows)


Chunk 1 traité, lignes dans ce chunk : 10000
Chunk 2 traité, lignes dans ce chunk : 10000
Chunk 3 traité, lignes dans ce chunk : 10000
Chunk 4 traité, lignes dans ce chunk : 10000
Chunk 5 traité, lignes dans ce chunk : 10000
Chunk 6 traité, lignes dans ce chunk : 10000
Chunk 7 traité, lignes dans ce chunk : 10000
Chunk 8 traité, lignes dans ce chunk : 10000
Chunk 9 traité, lignes dans ce chunk : 10000
Chunk 10 traité, lignes dans ce chunk : 10000
Chunk 11 traité, lignes dans ce chunk : 10000
Chunk 12 traité, lignes dans ce chunk : 10000
Chunk 13 traité, lignes dans ce chunk : 10000
Chunk 14 traité, lignes dans ce chunk : 10000
Chunk 15 traité, lignes dans ce chunk : 10000
Chunk 16 traité, lignes dans ce chunk : 10000
Chunk 17 traité, lignes dans ce chunk : 10000
Chunk 18 traité, lignes dans ce chunk : 10000
Chunk 19 traité, lignes dans ce chunk : 10000
Chunk 20 traité, lignes dans ce chunk : 10000
Chunk 21 traité, lignes dans ce chunk : 10000
Chunk 22 traité, lignes dans ce chunk : 100

In [5]:
from IPython.display import display

results = null_counts.to_frame(name="null_count").reset_index()
results.rename(columns={"index": "column"}, inplace=True)

results["null_percentage"] = results["null_count"] / total_rows * 100

results_sorted = results.sort_values(by="null_count", ascending=False)

print("Top 10 des colonnes avec le plus de valeurs nulles :")
display(results_sorted.head(10))


Top 10 des colonnes avec le plus de valeurs nulles :


,column,null_count,null_percentage
227,math_q16_average_score,586044,100.0
226,math_q15_average_score,586044,100.0
281,math_q15_total_timing,586044,100.0
268,math_q2_total_timing,586044,100.0
269,math_q3_total_timing,586044,100.0
282,math_q16_total_timing,586044,100.0
283,math_q17_total_timing,586044,100.0
284,math_q18_total_timing,586044,100.0
285,math_q19_total_timing,586044,100.0
272,math_q6_total_timing,586044,100.0


In [ ]:
output_stats_path = Path.home() / "hi-paris-2025/data/processed/X_test_null_stats.csv"
output_stats_path.parent.mkdir(parents=True, exist_ok=True)

results_sorted.to_csv(output_stats_path, index=False)
print("Statistiques de valeurs nulles sauvegardées dans :")
print(output_stats_path)

Statistiques de valeurs nulles sauvegardées dans :
/mnt/hdd/homes/akok/hi-paris-2025/data/processed/X_test_null_stats.csv


In [13]:
csv_path = Path.home() / "hi-paris-2025/data/processed/X_train_clean.csv"

df_header = pd.read_csv(csv_path, nrows=0)

print("Nombre de colonnes :", len(df_header.columns))
print("Noms des colonnes :")
print(list(df_header.columns))


Nombre de colonnes : 238
Noms des colonnes :
['Unnamed: 0', 'Year', 'CNT', 'CNTRYID', 'CYC', 'NatCen', 'STRATUM', 'SUBNATIO', 'OECD', 'ADMINMODE', 'LANGTEST_QQQ', 'LANGTEST_COG', 'LANGTEST_PAQ', 'Option_CT', 'Option_FL', 'Option_ICTQ', 'Option_WBQ', 'Option_PQ', 'Option_TQ', 'Option_UH', 'ST001D01T', 'ST003D02T', 'ST003D03T', 'ST004D01T', 'EFFORT1', 'EFFORT2', 'OCOD1', 'OCOD2', 'OCOD3', 'AGE', 'GRADE', 'ISCEDP', 'IMMIG', 'COBN_S', 'MISSSC', 'MATHEASE', 'WB153', 'ST253', 'ST311', 'WB165', 'IC171', 'ST038', 'IC180', 'ST255', 'ST350', 'ST034', 'FL162', 'PA195', 'ST330', 'FL169', 'ST230', 'ST331', 'ST354', 'PA042', 'ST305', 'ST258', 'PA167', 'IC176', 'ST352', 'IC173', 'PA188', 'ST283', 'ST256', 'WB154', 'WB166', 'FL150', 'ST266', 'IC174', 'ST005', 'ST021', 'IC182', 'FL164', 'FL160', 'ST267', 'ST338', 'ST351', 'ST342', 'ST355', 'FL166', 'PA183', 'ST007', 'FL170', 'WB168', 'WB163', 'IC184', 'ST348', 'ST226', 'ST336', 'WB160', 'FL167', 'PA166', 'ST250', 'ST268', 'IC177', 'PA196', 'IC183', 'ST

In [9]:
csv_path = Path.home() / "hi-paris-2025/data/processed/X_train_clean.csv"  
chunk_size = 100_000             

row_count = 0

for chunk in pd.read_csv(csv_path, chunksize=chunk_size):
    row_count += len(chunk)

print(f"Nombre total de lignes : {row_count}")


Nombre total de lignes : 1172086
